# Making a cubic crystal

Let's make a very tiny crystal: 27 particles to make a full cubic aggregate

In [1]:
import numpy as np
from json_dump import *
import contact_utils as cu
import config as cfg
from pathlib import Path
from geometry.cubic import CubicGeometry

In [2]:
### Key parameter: how we will name this run
run_name = "01_crystal"

### Define model parameters

model_params = {}

# ---------- LATTICE OPTIONS ----------

# Options:
# "chain", "square", "triangular", "cubic", "bcc", "fcc"
model_params["lattice_name"] = "cubic"

# Lattice dimensions
model_params["lx"] = 10
model_params["ly"] = 10  # Has to be 1 for chain
model_params["lz"] = 10  # Has to be 1 for square & triangular

In [3]:
# ---------- MODEL PARAMETERS ----------

# Number of particle types
model_params["n_types"] = 1

# Number of particles of each type
model_params["n_particles"] = [3**3]

# Couplings is its own beast. Should be gotten with the appropriate helper
# function.

## Important note: for 3D particles, do not use the get_matrix and set_contact utils!

A lot of face pairs are equivalent under rotation for cubes (e.g. (0, 0), (1, 3), (2, 2), (3, 1)).
When using a full contact matrix and feeding it to the wrapper using `set_single_species_contacts`, if all the equivalent pairs don't
have the same matrix coefficient, some of them will be overwritten by other values, and the contacts will be wrong.

Better to do things the way they are done below

In [4]:
get_opposite_face = CubicGeometry().get_opposite_face
# First, create a cmap_wrapper object with the appropriate number of particle types
cmap_wrapper = cu.ContactMapWrapper.cubic(model_params["n_types"])

# To make an ordered crystal, we want each face to touch its opposite on the cube.
# The opposite face to each face is given in the cube geometry.
# Actually, if I have gotten the code right, I only need one favorable contact per face (i.e. without counting the C4X(-) rotations.
# Let's test that!
interaction_e = -100.
# cmap_wrapper[0, 0] = interaction_e

# cmap_wrapper.contact_map[0] = interaction_e
# contact_31_coeff = 3 + 24 * 1
# cmap_wrapper.contact_map[contact_31_coeff] = interaction_e
# contact_22_coeff = 2 + 24 * 2
# cmap_wrapper.contact_map[contact_22_coeff] = interaction_e
# contact_13_coeff = 1 + 24 * 3
# cmap_wrapper.contact_map[contact_13_coeff] = interaction_e

for i in range(3):
    face = 4 * i
    opposite_face = get_opposite_face(face)
    cmap_wrapper[face, opposite_face] = interaction_e

# And process the couplings into flattened form
model_params["couplings"] = cmap_wrapper.get_formatted_couplings()

In [10]:
# Initialization option
model_params["initialize_option"] = "random"

# If "initialize_option" is set to "from_file", we must specify the location of
# the input file
# model_params["state_input"] = str(cfg.structures_path/"final_structure.dat")

# Options for average collection

model_params["state_av_option"] = True
model_params["e_av_option"] = True
model_params["e_record_option"] = False

if model_params["state_av_option"]:
    state_path = Path("./data/"+run_name+"/average_state")
    state_path.mkdir(parents = True, exist_ok = True)
    model_params["state_av_output"] =  str(state_path.resolve()) + "/"

if model_params["e_av_option"]:
    energy_path = Path("./data/"+run_name+"/average_energy")
    energy_path.mkdir(parents = True, exist_ok = True)
    model_params["e_av_output"] = str(energy_path.resolve())  + "/"

In [11]:
# Pick the probabilities of different moves. Has to sum to 1.
# Options:
""""
    "swap_empty_full",
    "swap_full_full",
    "rotate",
    "mutate",
    "rotate_and_swap_w_empty"
"""

moves_dict = {}
moves_dict["swap_empty_full"] = 1/3
moves_dict["rotate"] = 1/3
moves_dict["rotate_and_swap_w_empty"] = 1/3

model_params["move_probas"] = moves_dict

make_json_file(model_params, cfg.input_path/"model_params.json")

In [12]:
### Define mc parameters

mc_params = {}

# Number of MC steps used for equilibration
mc_params["mcs_eq"] = 1000

# Number of MC steps used for averaging
mc_params["mcs_av"] = 10

# Type of cooling schedule
# if exponential chosen: specify log10(T) as initial and final temperatures
mc_params["cooling_schedule"] = "exponential"

# Initial annealing temperature
mc_params["Ti"] = np.log10(30)

# Final annealing temperature
mc_params["Tf"] = 0

# Number of annealing steps
mc_params["Nt"] = 10

# Option to collect state checkpoints at the end of each temperature cycle
mc_params["checkpoint_option"] = True

# If checkpoint is True, we need to provide the output address for the 
# checkpoint files
structures_path = Path("./data/"+run_name+"/structures")
structures_path.mkdir(parents = True, exist_ok = True)

if mc_params["checkpoint_option"]:
    mc_params["checkpoint_address"] = str(structures_path.resolve())+"/"

# Output location of the final state configuration (must end with "/")
mc_params["final_structure_address"] = str(structures_path.resolve())+"/"

make_json_file(mc_params, cfg.input_path/"mc_params.json")

In [13]:
# Run the simulation from inside the notebook.
# Note that the program only writes to the cell output after it is done running.
# If you want to follow the excecution in real time, run the program from the command line!

# Parameter in run_sim is a boolean flag, which will overwrite the simulation results if set to True

cfg.run_simulation(overwrite = True)

At least one of the output folders is already populated!
overwrite flag set to True: running anyway.
Program runs!
1000
Printing current system state
Number of particle types: 1
Number of particles of each type: {27, }
Printing interactions structure
, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [17]:
# Plotting module creates a blender file which you have to open yourself
from plotting.plot_cubic import plot_cubes_from_simulation_results

blend_file_path = "./3dFigures/" + run_name + ".blend"
plot_cubes_from_simulation_results(struct_folder = str(structures_path.resolve()), fig_file = blend_file_path)

27
Info: Saved "01_crystal.blend"


Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_number